In [0]:
%matplotlib inline
!pip install livelossplot

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from skimage import io, transform
from tqdm import tqdm_notebook
from livelossplot import PlotLosses
from datetime import datetime
# Ignore warnings
import warnings,os
warnings.filterwarnings("ignore")


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()

'/device:GPU:0'

In [0]:
from google.colab import drive
drive.mount('/Bengali_Dataset',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /Bengali_Dataset


In [0]:
!ls "/Bengali_Dataset/My Drive/Bengali"


images_unzipped.csv	model.pth      train.csv     train.zip
images_unzipped.gsheet	pytorch.ipynb  train_subset


In [0]:
root_path = '/Bengali_Dataset/My Drive/Bengali/'
print(type(root_path))

<class 'str'>


In [0]:
df=pd.read_csv(root_path + "train.csv")
df.head()

,image_id,grapheme_root,vowel_diacritic,consonant_diacritic,grapheme
0,Train_0,15,9,5,ক্ট্রো
1,Train_1,159,0,0,হ
2,Train_2,22,3,5,খ্রী
3,Train_3,53,2,2,র্টি
4,Train_4,71,9,5,থ্রো


In [0]:
label1_num = df['grapheme_root'].max()+1
label2_num = df['vowel_diacritic'].max()+1
label3_num = df['consonant_diacritic'].max()+1

print(label1_num)
print(label2_num)
print(label3_num)

168
11
7


In [0]:
mean = 0.06922848809290576
std = 0.20515700083327537

In [0]:
class ToTensor(object):
    """Convert ndarrays in sample to Tensors. Also normalize the images"""

    def __call__(self, sample):
        image, labels = sample['image'], sample['labels']
        image = (image - 255)/255
        image = (image - mean)/std
        image = torch.from_numpy(image)
        
        image = image[np.newaxis,...]
        return {'image': image,
                'labels': torch.from_numpy(np.array(labels))}

In [0]:
csv_dir="/Bengali_Dataset/My Drive/Bengali/train.csv"
root_dir=root_path

In [0]:
class bengalidataset(Dataset):
    def __init__(self,csv_dir,root_dir,transform=None):
        self.csv_dir = csv_dir
        self.olddata = pd.read_csv(csv_dir)
        self.data = []
        self.root_dir = root_dir
        for index in range(len(self.olddata)):
          # print(index)
          filename= f'{self.root_dir}train_subset/train/{self.olddata.iloc[index,0]}.png'
          if(not os.path.exists(filename)): continue
          
          self.data.append(list(self.olddata.iloc[index]))

        self.data = pd.DataFrame(self.data,columns=['Image ID','label1','label2','label3','Grapheme'])

        print(self.data.shape)
        #self.data.to_csv('/Bengali_Dataset/My Drive/Bengali/images_unzipped.csv') #Don't forget to add '.csv' at the end of the path

        #print(self.data.head())

        self.transform = transform
        
        
    def __len__(self):
        return len(self.data)
        
    def __getitem__(self,index=0):
        '''
        Returns image, 
            
            label is,
            
            grapheme_root          15
            vowel_diacritic         9
            consonant_diacritic     5
        '''
        
        if torch.is_tensor(index):
            index = index.tolist()
            
        
        img_name = Path(f'{self.root_dir}train_subset/train/{self.data.iloc[index,0]}.png')
        img = io.imread(img_name)
        labels = self.data.iloc[index,1:4]
        
        labels = list(labels)
        
        sample = {"image":img,"labels":labels}        
        
        
        if self.transform:
            sample = self.transform(sample)


        
        return sample

In [0]:
# Dataset creation for visualization purpose.
datast = bengalidataset(csv_dir=Path(root_path + "train.csv"),root_dir=root_path)


(1851, 5)


In [0]:
datast = bengalidataset(csv_dir=Path(root_path + "train.csv"),root_dir=root_path,transform=transforms.Compose([ToTensor()]))
print(type(datast))
testset=datast


(1851, 5)
<class '__main__.bengalidataset'>


In [0]:
bs=256

In [0]:

testloader = torch.utils.data.DataLoader(testset, batch_size=bs,
                                         shuffle=True, num_workers=2)

In [0]:
class NNet(nn.Module):
    def __init__(self,label1_num,label2_num,label3_num):
        super(NNet,self).__init__()
        self.conv1 = nn.Conv2d(1,32,5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32,64,5)
        self.conv3 = nn.Conv2d(64,128,5)    
        self.fc1 = nn.Linear(128*12*12,label1_num)
        self.fc2 = nn.Linear(128*12*12,label2_num)
        self.fc3 = nn.Linear(128*12*12,label3_num)
        
        softmax = nn.Softmax(dim=1) 
    def forward(self,x):
        x = self.conv1(x)
        x = self.pool(x)
#         print(x.shape)
        x = F.relu(x)
        x = self.conv2(x)
        x = self.pool(x)
#         print(x.shape)
        x = F.relu(x)
        x = self.conv3(x)
        x = self.pool(x)
        bss = x.shape[0]     
        x = x.reshape(bss,-1)
        
        l1 = self.fc1(x)
        l2 = self.fc2(x)
        l3 = self.fc3(x)
        
        #print(l1.shape) #expecting batch_size*168, 8 for no.of images and 168 is count of grapheme_roots
        #print(l2.shape) #expecting batch_zsize*11
        #print(l3.shape) #expecting batchsize*7 

        grapheme_list=l1.cpu().data.numpy().argmax(axis=1)
        vowel_list=l2.cpu().data.numpy().argmax(axis=1)
        consonant_list=l3.cpu().data.numpy().argmax(axis=1)

        oldl1=l1
        oldl2=l2
        oldl3=l3


        
        return oldl1, oldl2, oldl3,grapheme_list,vowel_list,consonant_list

In [0]:
PATH=Path(root_path + "model.pth ")

In [0]:
# train on cuda if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

nnet = NNet(label1_num,label2_num,label3_num)
nnet.load_state_dict(torch.load('/Bengali_Dataset/My Drive/Bengali/model.pth'))

nnet.to(device)


NNet(
  (conv1): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=18432, out_features=168, bias=True)
  (fc2): Linear(in_features=18432, out_features=11, bias=True)
  (fc3): Linear(in_features=18432, out_features=7, bias=True)
)

In [0]:
nnet.fc1.weight


Parameter containing:
tensor([[-0.0024,  0.0033,  0.0054,  ...,  0.0084,  0.0135,  0.0025],
        [-0.0044, -0.0043,  0.0016,  ..., -0.0013, -0.0089, -0.0068],
        [-0.0014,  0.0056,  0.0033,  ..., -0.0051,  0.0084, -0.0048],
        ...,
        [ 0.0075, -0.0013, -0.0005,  ...,  0.0033,  0.0148,  0.0086],
        [-0.0006, -0.0007, -0.0048,  ...,  0.0023,  0.0077,  0.0003],
        [-0.0070, -0.0019,  0.0028,  ...,  0.0161,  0.0166,  0.0021]],
       device='cuda:0', requires_grad=True)

In [0]:
print(testset[1])
print(len(testset))
print(testset[i])
print(type(testset[1]))

{'image': tensor([[[-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183],
         [-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183],
         [-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183],
         ...,
         [-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183],
         [-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183],
         [-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183]]],
       dtype=torch.float64), 'labels': tensor([159,   0,   0])}
1851
{'image': tensor([[[-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183],
         [-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183],
         [-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183],
         ...,
         [-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183],
         [-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183],
         [-0.3183, -0.3183, -0.3183,  ..., -0.3183, -0.3183, -0.3183]]],
       dtype=torch.float64)

In [0]:
grapheme_overall=0
vowel_overall=0
consonant_overall=0
overall_overall=0

target_overall=0


for step,batch in tqdm_notebook(enumerate(testloader)):
  grapheme_counter=0
  vowel_counter=0
  consonant_counter=0
  overall_counter=0

  try:
    label1, label2, label3,grapheme_list,vowel_list,consonant_list = nnet(batch['image'].float().cuda())
  except Exception as e:
    print('working?', e)
  
  labels=batch['labels'].cuda()
  targ_label1=labels[:,0]
  targ_label2=labels[:,1]
  targ_label3=labels[:,2]
  
  target_overall=target_overall+len(grapheme_list)
  for i in range(len(grapheme_list)):
    if grapheme_list[i] == targ_label1[i]: ##confirm if targ_label 1 has all batc size numbers of just 1
      grapheme_counter=grapheme_counter+1
      grapheme_overall=grapheme_overall+1
    if vowel_list[i]==targ_label2[i]:
      vowel_counter=vowel_counter+1
      vowel_overall=vowel_overall+1
    if consonant_list[i]==targ_label3[i]:
      consonant_counter=consonant_counter+1
      consonant_overall=consonant_overall+1
    if grapheme_list[i] == targ_label1[i] and vowel_list[i]==targ_label2[i] and consonant_list[i]==targ_label3[i]:
      overall_counter=overall_counter+1
      overall_overall=overall_overall+1


  if(step>0):
      print("***.............................**")
      print(f"step is..{step}")
      print(f"count of images..{len(targ_label1)}")
      print(f"grapheme counter..{grapheme_counter}")
      print(f"grapheme_counter----  {round((grapheme_counter/len(targ_label1))*100,2)}%")
      print(f"vowel_counter----  {round((vowel_counter/len(targ_label1))*100,2)}%")
      print(f"consonant_counter----  {round((consonant_counter/len(targ_label1))*100,2)}%")
      print(f"overall_counter----  {round((overall_counter/len(targ_label1))*100,2)}%")

print("*****----------Overall----------------***************")
print(f"overall_counter...{overall_overall}")
print(f"overall image count...{target_overall}")
print(f"grapheme_overall_counter----  {round((grapheme_overall/(target_overall))*100,2)}%")
print(f"vowel_counter----  {round((vowel_overall/(target_overall))*100,2)}%")
print(f"consonant_counter----  {round((consonant_overall/(target_overall))*100,2)}%")
print(f"overall_counter----  {round((overall_overall/(target_overall))*100,2)}%")

***.............................**
step is..1
count of images..256
grapheme counter..240
grapheme_counter----  93.75%
vowel_counter----  95.31%
consonant_counter----  92.58%
overall_counter----  83.2%
***.............................**
step is..2
count of images..256
grapheme counter..239
grapheme_counter----  93.36%
vowel_counter----  96.09%
consonant_counter----  92.97%
overall_counter----  83.59%
***.............................**
step is..3
count of images..256
grapheme counter..234
grapheme_counter----  91.41%
vowel_counter----  95.7%
consonant_counter----  92.97%
overall_counter----  81.64%
***.............................**
step is..4
count of images..256
grapheme counter..239
grapheme_counter----  93.36%
vowel_counter----  94.92%
consonant_counter----  94.53%
overall_counter----  83.2%
***.............................**
step is..5
count of images..256
grapheme counter..239
grapheme_counter----  93.36%
vowel_counter----  96.48%
consonant_counter----  94.53%
overall_counter----  